# propaganda tweets scraping

In [6]:
import time
import tweepy
import pandas as pd
from IPython.display import clear_output
import pickle

In [2]:
# create connection with Twitter API

CONSUMER_KEY = 'J3uhwWHdmSt3uD69ry8r2kc3B'
CONSUMER_SECRET = 'LeuM7dVfFbXH0bc5fFkZIxlNiUXyjIx4Kcjh58HeWs2TrLijDx'
ACCESS_TOKEN = '327497511-5dNjiGaTQHASljoxNr1qlMOlHrrSB21HBFBDjx2E'
ACCESS_TOKEN_SECRET = 'tHLqwlhHYJWptW3femyq0rbMW6ZItu5yLQ3DFGJqJ8Xlg'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [3]:
prop_bots = pd.read_csv('data/propaganda_users.csv')['id'].tolist()

In [4]:
prop_bots = [int(x) for x in prop_bots]

In [8]:
tweets = []
cont, errors = 0, 0

for user in prop_bots:
    try:
        stuff = api.user_timeline(user_id = user, count = 100, include_rts = True, tweet_mode="extended")
        #tweet.is_quote_status indica se nel tweet c'è il riferimento ad un altro tweet

        for tweet in stuff:
            tweet._json['user_id'] = tweet._json['user']['id']
            if len(tweet._json['entities']['urls']) != 0:
                tweet._json['url'] = tweet._json['entities']['urls'][0]['expanded_url']
            else:
                 tweet._json['url'] = None
            del tweet._json['user'], tweet._json['entities']

            tweets.append(tweet._json)
    except:
        errors += 1
    cont += 1
    clear_output()
    print(str(cont) + ' / 3315   ---   errors: ' + str(errors))
    
    if cont % 500 == 0:
        path = 'propaganda_tweets_' + str(cont-500) + '-' + str(cont) + '.tw'
        pickle.dump( tweets, open( path , "wb" ) )
        tweets = []
        print 'sleeping...'
        time.sleep(900)

3312 / 3315   ---   errors: 11


In [12]:
path = 'propaganda_tweets_3000-3312.tw'
pickle.dump( tweets, open( path , "wb" ) )

In [ ]:
tweets_df = pd.DataFrame.from_dict(tweets)

In [ ]:
# rimuove tutte le interruzioni (spero)

tweets_df['full_text'] = tweets_df['full_text'].apply(lambda x: x.replace('\r', ' '))
tweets_df['full_text'] = tweets_df['full_text'].apply(lambda x: x.replace('\n', ' '))
tweets_df['full_text'] = tweets_df['full_text'].apply(lambda x: x.replace('\t', ' '))

In [ ]:
tweets_df.to_csv('data/propaganda_tweets.csv', index=False, sep='\t', encoding='utf-8-sig')